In [1]:
from matplotlib import pyplot as plt
from math import cos, sin, pi

import numpy as np
import SimpleITK as sitk
from gvxrPython3 import gvxr

import ipywidgets as widgets
from IPython.display import display, clear_output


Mon May 13 17:37:13 2024 (WW) Spekpy is not installed, try Xpecgen instead.
Mon May 13 17:37:13 2024 (WW) Xpecgen is not installed either.


In [2]:
# Create an OpenGL context
print("Create an OpenGL context")
gvxr.createOpenGLContext();

Mon May 13 17:37:13 2024 ---- Create window (ID: -1)
Mon May 13 17:37:13 2024 ---- Request an interactive OpenGL context
Mon May 13 17:37:13 2024 ---- Initialise GLFW
Mon May 13 17:37:13 2024 ---- Create an OpenGL window with a 3.2 context.
Mon May 13 17:37:13 2024 ---- Make the window's context current
Mon May 13 17:37:13 2024 ---- Initialise GLEW
Mon May 13 17:37:13 2024 ---- OpenGL vendor: NVIDIA Corporation
Mon May 13 17:37:13 2024 ---- OpenGL renderer: NVIDIA GeForce RTX 4060 Ti/PCIe/SSE2
Mon May 13 17:37:13 2024 ---- OpenGL version: 3.2.0 NVIDIA 545.23.08
Mon May 13 17:37:13 2024 ---- Use OpenGL 4.5.
Mon May 13 17:37:13 2024 ---- Initialise the X-ray renderer if needed and if possible


Create an OpenGL context


In [3]:
# Create a source
print("Set up the beam")
gvxr.setSourcePosition(-40.0,  0.0, 0.0, "cm");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

Set up the beam


In [4]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.setMonoChromatic(0.08, "MeV", 1000);
# The following is equivalent: gvxr.setMonoChromatic(80, "keV", 1000);

In [5]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm");
gvxr.setDetectorUpVector(0, 0, -1);
gvxr.setDetectorNumberOfPixels(640, 320);
gvxr.setDetectorPixelSize(0.5, 0.5, "mm");

Set up the detector


Mon May 13 17:37:14 2024 ---- Initialise the renderer


In [6]:
number_of_voxels = np.array([150, 150, 150])
spacing = np.array([0.5, 0.5, 0.5])

volume_size = number_of_voxels * spacing

iso_value = 4.5
a_value = 20
b_value = 15

In [7]:
gvxr.destroyImplicitSurface("points");
gvxr.createImplicitSurface("points");

gvxr.addControlPoint("points", 
    0, -8, 0,
    "mm",
    "META_BALL",
    a_value, b_value);

gvxr.addControlPoint("points", 
    0, 8, 0,
    "mm",
    "META_BALL",
    a_value, b_value);

Mon May 13 17:37:14 2024 (WW) ImplicitSurface points not found in g_implicit_surface_set.


In [8]:
gvxr.destroyImplicitSurface("lines");
gvxr.createImplicitSurface("lines");

gvxr.addControlLineSegment("lines", 
    -20, 0, 0, 20, 0, 0,
    "mm",
    "META_BALL",
    a_value, b_value);

gvxr.addControlLineSegment("lines", 
    0, -20, 0, 0, 20, 0,
    "mm",
    "META_BALL",
    a_value, b_value);


gvxr.addControlLineSegment("lines", 
    0, 0, -20, 0, 0, 20,
    "mm",
    "META_BALL",
    a_value, b_value);

Mon May 13 17:37:14 2024 (WW) ImplicitSurface lines not found in g_implicit_surface_set.


In [9]:
def voxelise(a_value, b_value):
    global volume_points, volume_lines

    
    for implicit_surface in gvxr.getImplicitSurfaceLabelSet():

        for point_id in range(gvxr.getNumberOfControlPoints(implicit_surface)):
            gvxr.setControlPointParameters(implicit_surface, point_id, "META_BALL", a_value, b_value);
                              
        for line_id in range(gvxr.getNumberOfControlLineSegments(implicit_surface)):
            gvxr.setControlLineSegmentParameters(implicit_surface, line_id, "META_BALL", a_value, b_value);

        gvxr.voxelise(implicit_surface,
                      number_of_voxels[0], number_of_voxels[1], number_of_voxels[2],
                      0, 0, 0,
                      spacing[0], spacing[1], spacing[2],
                      "mm");

In [10]:
def updateVolumeCallback(s):
    global volume_points, volume_lines
    voxelise(a_slider.value, b_slider.value)
    updateMeshCallback(iso_value_slider.value)


def updateMeshCallback(t):
    
    iso_value = iso_value_slider.value

    gvxr.removePolygonMeshesFromSceneGraph()

    gvxr.createMeshFromImplicitSurface("points", iso_value);
    gvxr.setColor("points", 0.0, 1.0, 0.0, 0.5)

    gvxr.translateNode("points", 0, 0, 5, "cm");
    gvxr.setMixture("points", "Ti90Al6V4")
    gvxr.setDensity("points", 4.43, "g/cm3")
    gvxr.addPolygonMeshAsInnerSurface("points")    

    gvxr.createMeshFromImplicitSurface("lines", iso_value);
    gvxr.setColor("lines", 1.0, 0.0, 0.0, 0.5)
    gvxr.translateNode("lines", 0, 0, -5, "cm");
    gvxr.setMixture("lines", "Ti90Al6V4")
    gvxr.setDensity("lines", 4.43, "g/cm3")
    gvxr.addPolygonMeshAsInnerSurface("lines")    

    # Update the visualisation window
    gvxr.displayScene()

    # Compute an X-ray image
    # We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
    try:
        if gvxr.getNumberOfPrimitives("points") or gvxr.getNumberOfPrimitives("lines"):
            gvxr.computeXRayImage();

            # Update the visualisation window
            gvxr.displayScene()

    except:
        print("An exception occurred")

    with output:
        output.clear_output()

        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.imshow(gvxr.takeScreenshot())

        plt.show()

In [11]:
%matplotlib widget
fig = None
ax = None

min_iso_value = 0
max_iso_value = a_value
iso_value_slider = widgets.FloatSlider(min=min_iso_value, max=max_iso_value, step=(max_iso_value - min_iso_value) / 100.0, value=iso_value)

a_slider = widgets.FloatSlider(min=1, max=40, step=0.1, value=a_value)
b_slider = widgets.FloatSlider(min=1, max=20, step=0.1, value=b_value)

output = widgets.Output(layout=widgets.Layout(height='600px'))

In [12]:
# Generate a previous
updateVolumeCallback(None)

In [13]:
gvxr.displayScene()
gvxr.setZoom(687.4293212890625)
gvxr.setSceneRotationMatrix((-0.4380485713481903,
     0.2018914669752121,
     -0.8759871125221252,
     0.0,
     -0.8969288468360901,
     -0.03282521665096283,
     0.4409559369087219,
     0.0,
     0.06027056649327278,
     0.9788579344749451,
     0.19546110928058624,
     0.0,
     0.0,
     0.0,
     0.0,
     1.0)
)
gvxr.displayScene()

In [14]:
with output:
    output.clear_output()

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(gvxr.takeScreenshot())

    plt.show()

In [15]:
# gvxr.renderLoop()

In [16]:
display(widgets.HBox((a_slider,b_slider,iso_value_slider)))
display(output)

a_slider.observe(updateVolumeCallback, 'value')
b_slider.observe(updateVolumeCallback, 'value')
iso_value_slider.observe(updateMeshCallback, 'value')

Output(layout=Layout(height='600px'))

In [17]:
voxelise(a_slider.value, b_slider.value)

volume_points = np.array(gvxr.getLastVoxelisation("points"));
volume_lines = np.array(gvxr.getLastVoxelisation("lines"));

In [18]:
sitk_image = sitk.GetImageFromArray(volume_points);
sitk_image.SetSpacing(spacing);
sitk.WriteImage(sitk_image, "output_data/implicit_function-points.mha")

In [19]:
sitk_image = sitk.GetImageFromArray(volume_lines);
sitk_image.SetSpacing(spacing);
sitk.WriteImage(sitk_image, "output_data/implicit_function-lines.mha")

In [20]:
gvxr.saveSTLfile("lines", "output_data/lines.stl")
gvxr.saveSTLfile("points", "output_data/points.stl")